In [ ]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import os
from datetime import datetime, timedelta

# Step 1: Create Directory to Save Images and Labels
output_dir = "spectrograms_train"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

label_file = os.path.join(output_dir, "labels.csv")

# Step 2: Download Stock Data with Extended Periods
def download_data_with_extension(ticker, start_date, end_date, chunk_size_days, extension_days):
    """
    Download time series data for a given ticker, including an extension for future trend analysis.
    """
    current_date = start_date
    all_data = []

    while current_date < end_date:
        next_date = current_date + timedelta(days=chunk_size_days + extension_days)
        print(f"Downloading {ticker} data from {current_date} to {next_date}...")
        
        data = yf.download(ticker, start=current_date, end=next_date, progress=False)
        if not data.empty:
            all_data.append(data['Close'])
        
        current_date = current_date + timedelta(days=chunk_size_days)

    return pd.concat(all_data, axis=0)

# Step 3: Compute Future Trend Label
def compute_future_trend_label(future_period_prices, threshold=0.005):
    """
    Compute label based on the average price variation between current and future periods:
        1 -> Positive trend (average price increase > threshold)
        0 -> Neutral trend (within ±threshold)
       -1 -> Negative trend (average price decrease > threshold)
    """
    if len(future_period_prices) == 0:
        return None
    
    price_change = (future_period_prices[-1]-future_period_prices[0])/future_period_prices[0]

    if price_change > threshold:
        return 1  # Upwards trend
    elif price_change < -threshold:
        return -1  # Downwards trend
    else:
        return 0  # Neutral trend

# Step 4: Generate Wavelet Spectrogram and Save Image
def generate_wavelet_spectrogram(signal, stock_name, period_num, wavelet='morl', scales=None):
    """
    Generate and save a spectrogram image using Continuous Wavelet Transform (CWT).
    """
    if scales is None:
        scales = np.arange(1, 128)

    # Perform Continuous Wavelet Transform
    coefficients, frequencies = pywt.cwt(signal, scales, wavelet)
    
    # Plot the spectrogram
    plt.figure(figsize=(10, 6))
    plt.imshow(np.abs(coefficients), extent=[0, len(signal), 1, 128],
               interpolation='bilinear', aspect='auto', cmap='jet')
    plt.colorbar(label='Power')
    plt.ylabel('Frequency (Scale)')
    plt.xlabel('Time')
    plt.title(f'{stock_name} - Period {period_num} Spectrogram')

    # Save the image
    image_path = os.path.join(output_dir, f"{stock_name}_Period{period_num}.png")
    plt.savefig(image_path)
    plt.close()
    print(f"Saved spectrogram: {image_path}")

    return image_path


# Step 5: Process Stocks and Save Labels
def process_stocks(tickers, start_date, end_date, chunk_size_days=100, extension_days=30, threshold=0.005):
    """
    Download stock data, generate spectrograms, and assign labels based on future trends.
    """
    labels = []

    for ticker in tickers:
        print(f"Processing {ticker}...")
        try:
            # Download data with extension for future trend analysis
            close_prices = download_data_with_extension(ticker, start_date, end_date, chunk_size_days, extension_days)
            close_prices = close_prices.dropna()
            
            if len(close_prices) == 0:
                print(f"No valid data for {ticker}. Skipping...")
                continue
            
            # Process each 100-day period
            period_num = 1
            for i in range(0, len(close_prices) - chunk_size_days - extension_days + 1, chunk_size_days):
                current_chunk = close_prices.iloc[i:i + chunk_size_days].values
                future_chunk = close_prices.iloc[i + chunk_size_days:i + chunk_size_days + extension_days].values
                
                if len(current_chunk) < 2 or len(future_chunk) < 2:
                    continue
                
                # Compute label based on future trend
                label = compute_future_trend_label( future_chunk, threshold)
                if label is None:
                    continue
                
                # Generate and save spectrogram
                image_path = generate_wavelet_spectrogram(current_chunk, ticker, period_num)
                
                # Append to labels list
                labels.append([image_path, label])
                period_num += 1
                
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    # Save labels to CSV
    labels_df = pd.DataFrame(labels, columns=["image_path", "label"])
    labels_df.to_csv(label_file, index=False)
    print(f"Labels saved to {label_file}")


# Step 6: Main Execution
if __name__ == "__main__":
    # List of Stocks (First 10 for demonstration)
    tickers = [ "AAPL", "MSFT", "AMZN", "GOOG", "BRK-B", "NVDA", "JNJ", "JPM", "PG", "UNH", "HD", "MA", "DIS", "NFLX", "KO", "PEP",
    "CSCO", "XOM", "PFE", "MRK", "INTC", "BA", "WMT", "MCD", "IBM", "GS",
    "NKE", "ADBE", "HON", "AMGN", "T", "COST", "ABT", "LMT", "BMY", "CAT",
    "DHR", "MDT", "CVX", "ORCL", "CMCSA", "MS", "ISRG", "TXN", "UNP", "QCOM",
    "LIN", "PM", "SBUX", "AMAT", "MMM", "NEE", "BLK", "SPGI", "NOW", "GE",
    "LLY", "ZTS", "LOW", "TGT", "INTU", "ANTM", "MU", "FIS", "CSX", "VRTX",
    "GILD", "APD", "PLD", "ADI", "C", "DUK", "SYK", "USB", "SO", "SCHW",
    "MMC", "CL", "TJX", "BDX", "CI", "PNC", "CME", "NSC", "EW", "ETN",
    "ADI", "ROP", "FCX", "ICE", "TRV", "TMO", "GM", "AON", "FDX", "HUM"]
    
    # Define Date Range (2007 to 2017)
    start_date = datetime(2000, 1, 1)
    end_date = datetime(2014, 1, 1)
    
    # Process Stocks and Generate Spectrograms with Labels
    process_stocks(tickers, start_date, end_date)

Processing AAPL...
Saved spectrogram: spectrograms_train/AAPL_Period1.png
Saved spectrogram: spectrograms_train/AAPL_Period2.png
Saved spectrogram: spectrograms_train/AAPL_Period3.png
Saved spectrogram: spectrograms_train/AAPL_Period4.png
Saved spectrogram: spectrograms_train/AAPL_Period5.png
Saved spectrogram: spectrograms_train/AAPL_Period6.png
Saved spectrogram: spectrograms_train/AAPL_Period7.png
Saved spectrogram: spectrograms_train/AAPL_Period8.png
Saved spectrogram: spectrograms_train/AAPL_Period9.png
Saved spectrogram: spectrograms_train/AAPL_Period10.png
Saved spectrogram: spectrograms_train/AAPL_Period11.png
Saved spectrogram: spectrograms_train/AAPL_Period12.png
Saved spectrogram: spectrograms_train/AAPL_Period13.png
Saved spectrogram: spectrograms_train/AAPL_Period14.png
Saved spectrogram: spectrograms_train/AAPL_Period15.png
Saved spectrogram: spectrograms_train/AAPL_Period16.png
Saved spectrogram: spectrograms_train/AAPL_Period17.png
Saved spectrogram: spectrograms_train/


1 Failed download:
['GOOG']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2000-01-01 00:00:00 -> 2000-05-10 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 946702800, endDate = 957931200")')



1 Failed download:
['GOOG']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2000-04-10 00:00:00 -> 2000-08-18 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 955339200, endDate = 966571200")')



1 Failed download:
['GOOG']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2000-07-19 00:00:00 -> 2000-11-26 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 963979200, endDate = 975214800")')



1 Failed download:
['GOOG']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2000-10-27 00:00:00 -> 2001-03-06 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 972619200, endDate = 983854800")')


...


1 Failed download:
['MA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2002-06-19 00:00:00 -> 2002-10-27 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1024459200, endDate = 1035691200")')



1 Failed download:
['MA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2002-09-27 00:00:00 -> 2003-02-04 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1033099200, endDate = 1044334800")')



1 Failed download:
['MA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2003-01-05 00:00:00 -> 2003-05-15 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1041742800, endDate = 1052971200")')



1 Failed download:
['MA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2003-04-15 00:00:00 -> 2003-08-23 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1050379200, endDate = 1061611200")')



1 Failed download:
['MA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2003-07-24 00:00:00 -> 2003-12-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1059019200, endDate = 1070254800")')


In [11]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import os
from datetime import datetime, timedelta

# Step 1: Create Directory to Save Images and Labels
output_dir = "spectrograms_test"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

label_file = os.path.join(output_dir, "labels.csv")

# Step 2: Download Stock Data with Extended Periods
def download_data_with_extension(ticker, start_date, end_date, chunk_size_days, extension_days):
    """
    Download time series data for a given ticker, including an extension for future trend analysis.
    """
    current_date = start_date
    all_data = []

    while current_date < end_date:
        next_date = current_date + timedelta(days=chunk_size_days + extension_days)
        print(f"Downloading {ticker} data from {current_date} to {next_date}...")
        
        data = yf.download(ticker, start=current_date, end=next_date, progress=False)
        if not data.empty:
            all_data.append(data['Close'])
        
        current_date = current_date + timedelta(days=chunk_size_days)

    return pd.concat(all_data, axis=0)

# Step 3: Compute Future Trend Label
def compute_future_trend_label(future_period_prices, threshold=0.005):
    """
    Compute label based on the average price variation between current and future periods:
        1 -> Positive trend (average price increase > threshold)
        0 -> Neutral trend (within ±threshold)
       -1 -> Negative trend (average price decrease > threshold)
    """
    if len(future_period_prices) == 0:
        return None
    
    price_change = (future_period_prices[-1]-future_period_prices[0])/future_period_prices[0]

    if price_change > threshold:
        return 1  # Upwards trend
    elif price_change < -threshold:
        return -1  # Downwards trend
    else:
        return 0  # Neutral trend

# Step 4: Generate Wavelet Spectrogram and Save Image
def generate_wavelet_spectrogram(signal, stock_name, period_num, wavelet='morl', scales=None):
    """
    Generate and save a spectrogram image using Continuous Wavelet Transform (CWT).
    """
    if scales is None:
        scales = np.arange(1, 128)

    # Perform Continuous Wavelet Transform
    coefficients, frequencies = pywt.cwt(signal, scales, wavelet)
    
    # Plot the spectrogram
    plt.figure(figsize=(10, 6))
    plt.imshow(np.abs(coefficients), extent=[0, len(signal), 1, 128],
               interpolation='bilinear', aspect='auto', cmap='jet')
    plt.colorbar(label='Power')
    plt.ylabel('Frequency (Scale)')
    plt.xlabel('Time')
    plt.title(f'{stock_name} - Period {period_num} Spectrogram')

    # Save the image
    image_path = os.path.join(output_dir, f"{stock_name}_Period{period_num}.png")
    plt.savefig(image_path)
    plt.close()
    print(f"Saved spectrogram: {image_path}")

    return image_path


# Step 5: Process Stocks and Save Labels
def process_stocks(tickers, start_date, end_date, chunk_size_days=100, extension_days=30, threshold=0.005):
    """
    Download stock data, generate spectrograms, and assign labels based on future trends.
    """
    labels = []

    for ticker in tickers:
        print(f"Processing {ticker}...")
        try:
            # Download data with extension for future trend analysis
            close_prices = download_data_with_extension(ticker, start_date, end_date, chunk_size_days, extension_days)
            close_prices = close_prices.dropna()
            
            if len(close_prices) == 0:
                print(f"No valid data for {ticker}. Skipping...")
                continue
            
            # Process each 100-day period
            period_num = 1
            for i in range(0, len(close_prices) - chunk_size_days - extension_days + 1, chunk_size_days):
                current_chunk = close_prices.iloc[i:i + chunk_size_days].values
                future_chunk = close_prices.iloc[i + chunk_size_days:i + chunk_size_days + extension_days].values
                
                if len(current_chunk) < 2 or len(future_chunk) < 2:
                    continue
                
                # Compute label based on future trend
                label = compute_future_trend_label( future_chunk, threshold)
                if label is None:
                    continue
                
                # Generate and save spectrogram
                image_path = generate_wavelet_spectrogram(current_chunk, ticker, period_num)
                
                # Append to labels list
                labels.append([image_path, label])
                period_num += 1
                
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    # Save labels to CSV
    labels_df = pd.DataFrame(labels, columns=["image_path", "label"])
    labels_df.to_csv(label_file, index=False)
    print(f"Labels saved to {label_file}")


# Step 6: Main Execution
if __name__ == "__main__":
    # List of Stocks (First 10 for demonstration)
    tickers = [ "AAPL", "MSFT", "AMZN", "GOOG", "BRK-B", "NVDA", "JNJ", "JPM", "PG", "UNH", "HD", "MA", "DIS", "NFLX", "KO", "PEP",
    "CSCO", "XOM", "PFE", "MRK", "INTC", "BA", "WMT", "MCD", "IBM", "GS",
    "NKE", "ADBE", "HON", "AMGN", "T", "COST", "ABT", "LMT", "BMY", "CAT",
    "DHR", "MDT", "CVX", "ORCL", "CMCSA", "MS", "ISRG", "TXN", "UNP", "QCOM",
    "LIN", "PM", "SBUX", "AMAT", "MMM", "NEE", "BLK", "SPGI", "NOW", "GE",
    "LLY", "ZTS", "LOW", "TGT", "INTU", "ANTM", "MU", "FIS", "CSX", "VRTX",
    "GILD", "APD", "PLD", "ADI", "C", "DUK", "SYK", "USB", "SO", "SCHW",
    "MMC", "CL", "TJX", "BDX", "CI", "PNC", "CME", "NSC", "EW", "ETN",
    "ADI", "ROP", "FCX", "ICE", "TRV", "TMO", "GM", "AON", "FDX", "HUM"]
    
    # Define Date Range (2007 to 2017)
    start_date = datetime(2016, 1, 1)
    end_date = datetime(2019, 1, 1)
    
    # Process Stocks and Generate Spectrograms with Labels
    process_stocks(tickers, start_date, end_date)

Processing AAPL...
Saved spectrogram: spectrograms_test/AAPL_Period1.png
Saved spectrogram: spectrograms_test/AAPL_Period2.png
Saved spectrogram: spectrograms_test/AAPL_Period3.png
Saved spectrogram: spectrograms_test/AAPL_Period4.png
Saved spectrogram: spectrograms_test/AAPL_Period5.png
Saved spectrogram: spectrograms_test/AAPL_Period6.png
Saved spectrogram: spectrograms_test/AAPL_Period7.png
Saved spectrogram: spectrograms_test/AAPL_Period8.png
Saved spectrogram: spectrograms_test/AAPL_Period9.png
Labels saved to spectrograms_test/labels.csv
